# OpenStrMap


In [1]:
#!pip install OSMPythonTools -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 2.7 MB/s eta 0:00:00


In [2]:
from OSMPythonTools.nominatim import Nominatim

In [3]:
from tqdm import tqdm

In [50]:
PLANTS_NAMES = [
    'Lespedeza bicolor',
'Digitalis lanata',
'Taraxacum officinale',
'Hylotelephium maximum',
'Tanacetum vulgare',
'Paeonia anomala',
'Abies sibirica',
'Plantago afra',
'Podophyllum peltatum',
'Artemisia absinthium'
]

In [51]:
import os
import pandas as pd

plant_regions = {}
folder_path = "/content/data"

file_list = os.listdir(folder_path)

for file_name in file_list:
    if file_name.endswith(".csv"):
        #Читаем файл
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path, delimiter='\t')

        to_check = {names:[] for names in PLANTS_NAMES}
        for lat, lon, scientificName in zip(df.decimalLatitude.to_list(), df.decimalLongitude.to_list(), df.scientificName.to_list()):
            #Ищем название растения в нашем списке и используем его
            for name in PLANTS_NAMES:
                if name in scientificName:
                    to_check[name].append((lat, lon))

        cleaned_dict = {}
        for key, value in to_check.items():
            unique_values = list(set(value))
            cleaned_dict[key] = unique_values
            print('Plant:', key, ', number coord:', len(unique_values))
        

<ipython-input-51-405b3ecabd30>:13: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter='\t')


Plant: Lespedeza bicolor , number coord: 780
Plant: Digitalis lanata , number coord: 1
Plant: Taraxacum officinale , number coord: 17321
Plant: Hylotelephium maximum , number coord: 1395
Plant: Tanacetum vulgare , number coord: 16957
Plant: Paeonia anomala , number coord: 0
Plant: Abies sibirica , number coord: 3723
Plant: Plantago afra , number coord: 1
Plant: Podophyllum peltatum , number coord: 0
Plant: Artemisia absinthium , number coord: 0


In [44]:
import yaml

with open(f'name_to_coord{file_name[:-4]}.yml', 'w', encoding = 'utf-8') as outfile:
    yaml.dump(cleaned_dict, outfile, default_flow_style=False, allow_unicode=True)

In [45]:
name_region_dict = {names:[] for names in PLANTS_NAMES}
nominatim = Nominatim()
for name, coords in cleaned_dict.items():
    
    for lat, lon in tqdm(coords):
        query_to_map = nominatim.query(lat, lon, reverse=True, zoom=8)
        try:
            state = query_to_map.address()['state']
        except:
            state = ''
        try:
            region = query_to_map.address()['region']
        except:
            region = ''

        name_region_dict[name].append((state, region))

Выходные данные были обрезаны до нескольких последних строк (5000).
100%|██████████| 2088/2088 [14:48<00:00,  2.35it/s]
0it [00:00, ?it/s]
100%|██████████| 5450/5450 [36:07<00:00,  2.51it/s]


In [ ]:
name_region_dict

In [48]:
plant_regions_results = {}
for key, value in name_region_dict.items():
    unique_values = list(set(value))
    plant_regions_results[key] = unique_values
    print('Plant:', key, ', number regions:', len(unique_values))

Plant: Bupleurum longifolium , number regions: 28
Plant: Hamamelis virginiana , number regions: 0
Plant: Datura stramonium , number regions: 52
Plant: Origanum vulgare , number regions: 73
Plant: Erysimum canescens , number regions: 38
Plant: Rhamnus cathartica , number regions: 57
Plant: Catharanthus roseus , number regions: 0
Plant: Hedysarum alpinum , number regions: 29
Plant: Frangula alnu , number regions: 66


In [49]:
import yaml

with open(f'name_to_region{file_name[:-4]}.yml', 'w', encoding = 'utf-8') as outfile:
    yaml.dump(plant_regions_results, outfile, default_flow_style=False, allow_unicode=True)

In [2]:
import yaml

In [6]:
with open("C:\\Users\\1bitt\\Documents\\projects\\MedPlantsGPT\\data\\name_to_region.yml", "r", encoding='utf-8') as stream:
    try:
        name_to_region = yaml.load(stream, Loader=yaml.Loader)
    except yaml.YAMLError as exc:
        print(exc)

In [9]:
def find_plants_in_region(region, regions_dict):
    plants_in_region = []
    
    # Перебираем все ключи и значения в словаре областей
    for plant, regions in regions_dict.items():
        # Перебираем области для каждого растения
        for r in regions:
            if r[0] == region:
                # Если область совпадает с заданной, добавляем растение в список
                plants_in_region.append(plant)
                break  # Прекращаем перебор областей для данного растения
    
    # Выводим результат
    if len(plants_in_region) > 0:
        print(f"Растения, растущие в области {region}:")
        for plant in plants_in_region:
            print(plant)
    else:
        print(f"В области {region} нет известных растений.")

# Пример словаря областей
regions_dict = {
    'Aconitum leucostomum': [
        ('Кемеровская область', 'Сибирский федеральный округ'),
        ('Республика Алтай', 'Сибирский федеральный округ'),
        ('Алтайский край', 'Сибирский федеральный округ'),
        ('Республика Тыва', 'Сибирский федеральный округ'),
        ('Республика Бурятия', 'Дальневосточный федеральный округ'),
        ('Красноярский край', 'Сибирский федеральный округ')
    ],
    'Acorus calamus': [
        ('Омская область', 'Сибирский федеральный округ'),
        ('Липецкая область', 'Центральный федеральный округ'),
        ('Калужская область', 'Центральный федеральный округ'),
        ('Кемеровская область', 'Сибирский федеральный округ')
    ]
}

# Пример использования функции
find_plants_in_region('Кемеровская область', regions_dict)


Растения, растущие в области Кемеровская область:
Aconitum leucostomum
Acorus calamus


In [ ]:
region = 'Кемеровская область'

if region in 